In [1]:
import nibabel as nib
import numpy as np
import os
from os import path
import pandas as pd
from scipy.spatial import distance_matrix
from sklearn.cluster import DBSCAN
import sys
from zipfile import ZipFile

In [2]:
from fsedit_notes_helpers import get_diff_data, summarize_edits, get_bm_edits, get_wm_edits, get_cp_edits, get_bfs_edits

In [4]:
subnum = 'CC0010'
typ = 'bm'
basedir='/Users/zeynepenkavi/Downloads/ConteQC'
outdir='/Users/zeynepenkavi/Downloads/ConteQC'

In [5]:
# Check if original zip exists. If not print error and quit
exists = path.isfile(path.join(basedir, 'sub-'+subnum+'.zip'))
if not exists:
    print("Unedited images not found in directory. Quitting...")
    try:
        sys.exit()
    except SystemExit:
        print("Quit because difference images cannot be calculated without unedited images.")
else:
    # Unzip unedited images of potential interest
    toExtract = ['brainmask.mgz', 'wm.mgz', 'brain.finalsurfs.mgz']
    suffix = path.join('sub-'+subnum, 'mri')
    toExtract = [path.join(suffix, s) for s in toExtract]
    with ZipFile(path.join(basedir, 'sub-'+subnum+'.zip'), 'r') as zipObj:
        # Get a list of all archived file names from the zip
        listOfFileNames = zipObj.namelist()
        # Iterate over the file names
        for fileName in listOfFileNames:
            # Check filename endswith csv
            if fileName in toExtract:
                # Extract a single file from zip
                zipObj.extract(fileName, path.join(basedir, 'sub-'+subnum+'_unedited'))
    print("Extracting unedited files complete.")

Extracting unedited files complete.


In [6]:
editp = path.join(basedir, 'sub-'+subnum)
uneditp = path.join(basedir, 'sub-'+subnum+'_unedited')
typ_dict = {'bm': 'brainmask', 'wm': 'wm', 'bfs': 'brain.finalsurfs', 'cp': 'control.dat'}
vol = typ_dict[typ]

In [7]:
if vol == "brain.finalsurfs":
    edit_img = nib.load(path.join(editp,'mri/%s.manedit.mgz')%(vol))
else:
    edit_img = nib.load(path.join(editp,'mri/%s.mgz')%(vol))

edit_data = edit_img.get_fdata()
unedit_img = nib.load(path.join(uneditp,'sub-%s/mri/%s.mgz')%(subnum, vol))
unedit_data = unedit_img.get_fdata()
diff_data = unedit_data - edit_data

In [9]:
out = pd.DataFrame(np.asarray(np.asarray(diff_data != 0).nonzero()).T).rename(columns={0:"Sag", 1:"Axe", 2:"Cor"})
out['diff_val'] = diff_data[np.where(diff_data != 0)]
out['Action'] = np.where(out.diff_val>0, "delete voxel", "add voxel")

In [11]:
out.query('Action == "add voxel"')

,Sag,Axe,Cor,diff_val,Action
41,77,58,124,-1.0,add voxel
66,78,57,124,-1.0,add voxel
131,80,55,99,-1.0,add voxel
135,80,55,124,-1.0,add voxel
153,81,54,99,-1.0,add voxel
186,82,52,100,-1.0,add voxel
187,82,53,99,-1.0,add voxel
221,83,51,100,-1.0,add voxel
260,84,51,100,-1.0,add voxel
261,84,52,99,-1.0,add voxel
